In [1]:
!pip install -U pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1


# 创建本地Milvus向量数据库

In [2]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

# 创建Collection

在milvus中，我们需要定义一个collection来存储vector和关联的metadata，collection可以理解为传统关系型数据库的table。

创建collection需要定义schema、index参数（比如向量维度、index类型和distant指标）。

这里进行最小设置：**collection名字**和**collection的向量维度**。

In [3]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
) # The default of metric type is "COSINE"，primary key is "id"，vector name is "vector"

# 文本表示为向量

In [4]:
# 安装模型所需的工具，如果没有安装pytorch，那么会下载pytorch
!pip install "pymilvus[model]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


使用默认Embedding模型生成向量，milvus需要的数据格式是一个list，其中每个元素都是一个数据记录

In [5]:
from pymilvus import model


embedding_fn = model.DefaultEmbeddingFunction()

docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = embedding_fn.encode_documents(docs)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

Dim: 768 (768,)
Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


# (optional)如果上述方法有网络问题，无法下载模型，可以使用如下随机方法来代替

In [ ]:
import random

docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]
vectors = [[random.uniform(-1, 1) for _ in range(768)] for _ in docs]
data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


# 把数据插入到collection中

In [6]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


# 语义搜索

In [7]:
query_vectors = embedding_fn.encode_queries(["Who is Alan Turing?"])

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)

data: ["[{'id': 2, 'distance': 0.5859944820404053, 'entity': {'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}}, {'id': 1, 'distance': 0.5118255019187927, 'entity': {'text': 'Alan Turing was the first person to conduct substantial research in AI.', 'subject': 'history'}}]"] 


# 使用metadata过滤进行向量搜索

milvus可以使用字段scalar进行元数据过滤，默认是scalar不打开，如果想打开scalar，请参考[index](https://milvus.io/docs/scalar_index.md)

In [8]:
docs = [
    "Machine learning has been used for drug design.",
    "Computational synthesis with AI algorithms predicts molecular properties.",
    "DDR1 is involved in cancers and fibrosis.",
]
vectors = embedding_fn.encode_documents(docs)
data = [
    {"id": 3 + i, "vector": vectors[i], "text": docs[i], "subject": "biology"}
    for i in range(len(vectors))
]

client.insert(collection_name="demo_collection", data=data)

res = client.search(
    collection_name="demo_collection",
    data=embedding_fn.encode_queries(["tell me AI related information"]),
    filter="subject == 'biology'",
    limit=2,
    output_fields=["text", "subject"],
)

print(res)


data: ["[{'id': 4, 'distance': 0.27030572295188904, 'entity': {'text': 'Computational synthesis with AI algorithms predicts molecular properties.', 'subject': 'biology'}}, {'id': 3, 'distance': 0.1642588973045349, 'entity': {'text': 'Machine learning has been used for drug design.', 'subject': 'biology'}}]"] 


# Query

query()可以检索匹配到特定规则的所有entities

In [9]:
# 检索scalar字段包括history的所有entities
res = client.query(
    collection_name="demo_collection",
    filter="subject == 'history'",
    output_fields=["text", "subject"],
)

In [10]:
res

data: ["{'id': 0, 'text': 'Artificial intelligence was founded as an academic discipline in 1956.', 'subject': 'history'}", "{'id': 1, 'text': 'Alan Turing was the first person to conduct substantial research in AI.', 'subject': 'history'}", "{'id': 2, 'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}"] 

In [11]:
# 直接按照primary key检索
res = client.query(
    collection_name="demo_collection",
    ids=[0, 2],
    output_fields=["vector", "text", "subject"],
)
res

data: ["{'id': 0, 'text': 'Artificial intelligence was founded as an academic discipline in 1956.', 'subject': 'history', 'vector': [0.010727894, -0.035895176, 0.018749746, 0.016348809, 0.03651688, 0.0035882127, -0.0004004981, 0.028529339, 0.0022745496, 0.0018362562, 0.004225859, 0.02717393, -0.0036843517, 0.03079148, 0.0045053945, 0.0442281, 0.010503842, -0.029494502, -0.06707345, -0.02052648, 0.015322829, -0.0060049067, -0.0622855, -0.039614785, 0.014206286, 0.032707665, -0.020834554, -0.044174287, -0.028339839, 0.029424427, -0.028087234, -0.020809026, 0.017159794, 0.0021116603, 0.021823755, -0.001577665, -0.037696734, 0.041460764, -0.02505642, 0.083336696, -0.015979158, 0.009813834, -0.026605438, 0.00061898713, 0.0037358247, -0.034155212, 0.058708236, -0.023721864, 0.0067459764, -0.03584181, -0.017560031, 0.022803081, 0.0026646194, 0.02511928, 0.046945184, 0.012622784, 0.018337477, -0.0071655777, 0.042811263, 0.0050310283, 0.05705552, -0.014866053, 0.10045472, 0.0064573605, -0.06832

# 删除entities

In [12]:
# Delete entities by primary key
res = client.delete(collection_name="demo_collection", ids=[0, 2])

print(res)

# Delete entities by a filter expression
res = client.delete(
    collection_name="demo_collection",
    filter="subject == 'biology'",
)

print(res)


[0, 2]
[3, 4, 5]


# 导入已有数据

由于milvus所有数据有存在了local file，因此，可以在程序中断后，就可以把所有数据导入到memory中，可以通过对已存在的数据创建MilvusClient来使用。例如，可以从"milvus_demo.db"文件恢复collection，并继续写入数据。

In [13]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")
client

# 删除collection

In [14]:
# Drop collection
client.drop_collection(collection_name="demo_collection")

# 生成环境最好使用[docker](https://milvus.io/docs/install_standalone-docker.md)或者[Kubernetes](https://milvus.io/docs/install_cluster-milvusoperator.md)
不同部署方案的API是相同的，因此不同部署方案切换后只需要调整URL和Token即可。

In [ ]:
client = MilvusClient(uri="http://localhost:19530", token="root:Milvus")
client

milvus提供REST和gRPC API，支持 Python, Java, Go, C# 和Node.js